**Table of Contents:**
- Introduction
- Architecture Overview
- ConvNet Layers
    - Convolutional Layer
    - Pooling Layer
    - Normalization Layer
    - Fully-Connected Layer
    - Converting Fully-Connected Layers to Convolutional Layers
- ConvNet Architectures
    - Layer Patterns
    - Layer Sizing Patterns
    - Case Studies (LeNet / AlexNet / ZFNet / GoogLeNet / VGGNet)
    - Computational Considerations
- Additional References

## Introduction: Convolutional Neural Networks (CNNs/ConvNets)

`Convolutional Neural Networks` are very similar to ordinary Neural Networks from the previous chapter: 
- They are made up of neurons that have learnable `weights` and `biases`. 
- Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. 
- The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other. 
- They have a loss function (e.g. SVM/Softmax) on the last (`fully-connected`) layer and all the tips/tricks we developed for learning regular Neural Networks still apply.

So what changes? ConvNet architectures make the explicit assumption that the **inputs are images**, which allows us to encode certain properties into the architecture. These then make the `forward function` more efficient to implement and vastly reduce the amount of parameters in the network.

## Architecture Overview

**Recall: Regular Neural Nets**. 
- Neural Networks receive an input (a single vector), and transform it through a series of hidden layers. 
- Each hidden layer is made up of a set of neurons, where each neuron is *fully connected* to all neurons in the previous layer, and where neurons in a single layer function completely independently and do not share any connections. 
- The last `fully-connected` layer is called the “output layer” and in classification settings it represents the class scores.

**Regular Neural Nets don’t scale well to full images**. In CIFAR-10, images are only of size 32x32x3 (32 wide, 32 high, 3 color channels), so a single fully-connected neuron in a first hidden layer of a regular Neural Network would have `32*32*3 = 3072` weights. This amount still seems manageable, but clearly this fully-connected structure does not scale to larger images. For example, an image of more respectable size, e.g. 200x200x3, would lead to neurons that have `200*200*3 = 120,000` weights. 
<br>Moreover, we would almost certainly want to have several such neurons, so the parameters would add up quickly! Clearly, **this full connectivity is wasteful and the huge number of parameters would quickly lead to overfitting**.

**3D volumes of neurons**. Convolutional Neural Networks take advantage of the fact that the input consists of images and they constrain the architecture in a more sensible way. 
<br>In particular, unlike a regular Neural Network, the layers of a ConvNet have neurons arranged in 3 dimensions: width, height, depth. (Note that the word depth here refers to the third dimension of an **activation volume**, not to the depth of a full Neural Network, which can refer to the total number of layers in a network.) 
<br>For example, the input images in CIFAR-10 are an input volume of activations, and the volume has dimensions 32x32x3 (width, height, depth respectively). As we will soon see, the neurons in a layer will only be **connected to a small region of the layer before it**, instead of all of the neurons in a fully-connected manner. 
<br>Moreover, the final output layer would for CIFAR-10 have dimensions 1x1x10, because by the end of the ConvNet architecture we will reduce the full image into a single vector of class scores, arranged along the depth dimension. 

Here is a visualization:

A regular 3-layer Neural Network|A ConvNet arranges its neurons in three dimensions (width, height, depth), as visualized in one of the layers
:-:|:-:
![](./Piks/neural_net2.jpeg)|![](./Piks/cnn.jpeg)

*Every layer of a ConvNet transforms the 3D input volume to a 3D output volume of neuron activations. In this example, the red input layer holds the image, so its width and height would be the dimensions of the image, and the depth would be 3 (Red, Green, Blue channels).*

> **A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters.**

## Layers used to build ConvNets

As we described above, a simple ConvNet is a sequence of layers, and every layer of a ConvNet transforms one volume of activations to another through a differentiable function. 
<br>We use **three** main types of layers to build ConvNet architectures: 
- **Convolutional Layer**.
- **Pooling Layer**.
- **Fully-Connected Layer** (exactly as seen in regular Neural Networks). 

We will stack these layers to form a full **ConvNet architecture**.

**Example Architecture: Overview**. We will go into more details below, but a simple ConvNet for CIFAR-10 classification could have the architecture **\[INPUT - CONV - RELU - POOL - FC\]**. In more detail:
- **INPUT** \[32x32x3\] will hold the **raw pixel values** of the image, in this case an image of width 32, height 32, and with three color channels R, G, B.
- **CONV layer** will **compute the output of neurons that are connected to local regions in the input**, each computing a dot product between their weights and a small region they are connected to in the input volume. This may result in volume such as \[32x32x12\] if we decided to use 12 filters.
- **RELU layer** will apply an **elementwise activation function**, such as the max(0, x) thresholding at zero. This leaves the size of the volume unchanged (\[32x32x12\]).
- **POOL layer** will perform a **downsampling** operation along the spatial dimensions (width, height), resulting in volume such as \[16x16x12\].
- **FC (fully-connected) layer** will compute the class scores, resulting in volume of size \[1x1x10\], where each of the 10 numbers correspond to a class score, such as among the 10 categories of CIFAR-10. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

In this way, ConvNets transform the original image layer by layer from the original pixel values to the final class scores. Note that **some layers contain parameters and other don’t**: 
- In particular, the `CONV/FC layers` perform transformations that are a function of not only the **activations** in the input volume, but also of the **parameters** (the weights and biases of the neurons). 
- On the other hand, the `RELU/POOL layers` will implement a **fixed function**. 

The parameters in the CONV/FC layers will be **trained with gradient descent** so that the class scores that the ConvNet computes are consistent with the labels in the training set for each image.

In summary:
- A ConvNet architecture is in the simplest case a list of Layers that transform the image volume into an output volume (e.g. holding the class scores)
- There are a few distinct types of Layers (e.g. CONV/FC/RELU/POOL are by far the most popular)
- Each Layer accepts an input 3D volume and transforms it to an output 3D volume through a differentiable function
- Each Layer may or may not have parameters (e.g. CONV/FC do, RELU/POOL don’t)
- Each Layer may or may not have additional hyperparameters (e.g. CONV/FC/POOL do, RELU doesn’t)

![](./Piks/convnet.jpeg)

*The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows. The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores, and print the labels of each one. The full web-based demo is shown in the header of our website. The architecture shown here is a tiny VGG Net, which we will discuss later.*

### Convolutional Layer

The Conv layer is the core building block of a Convolutional Network that does most of the computational heavy lifting.

**Overview and intuition without brain stuff.**
<br>Let’s first discuss what the CONV layer computes without brain/neuron analogies. 
- The CONV layer's parameters consist of **a set of learnable filters**. 
- Every filter is small spatially (along width and height), but extends through the full depth of the input volume. 

For example: 
- A typical filter on a first layer of a ConvNet might have size 5x5x3 (i.e. 5 pixels width and height, and 3 because images have depth 3, the color channels). 
- During the forward pass, we slide (more precisely, convolve) each filter across the width and height of the input volume and compute dot products between the entries of the filter and the input at any position. 
- As we slide the filter over the width and height of the input volume, we will produce a 2-dimensional activation map that gives the responses of that filter at every spatial position. 
- Intuitively, the network will learn filters that activate when they see some type of visual feature such as an edge of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the network. 
- Now, we will have an entire set of filters in each CONV layer (e.g. 12 filters), and each of them will produce a separate 2-dimensional activation map. We will stack these activation maps along the depth dimension and produce the output volume.

**The brain view.** 
<br>If you’re a fan of the brain/neuron analogies, every entry in the 3D output volume can also be interpreted as an output of a neuron that looks at only a small region in the input and shares parameters with all neurons to the left and right spatially (since these numbers all result from applying the same filter). We now discuss the details of the neuron connectivities, their arrangement in space, and their parameter sharing scheme.

**Local Connectivity.**
<br>When dealing with high-dimensional inputs such as images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. 
<br>Instead, we will connect each neuron to only **a local region** of the input volume. The spatial extent of this connectivity is a hyperparameter called the **receptive field of the neuron** (equivalently this is the filter size). The extent of the connectivity along the depth axis is always equal to the depth of the input volume. It is important to emphasize again this **asymmetry** in how we treat the spatial dimensions (width and height) and the depth dimension: **The connections are local in space (along width and height), but always full along the entire depth of the input volume.**

*Example 1.* 
<br>For example, suppose that the input volume has size \[32x32x3\], (e.g. an RGB CIFAR-10 image). If the receptive field (or the filter size) is 5x5, then each neuron in the Conv Layer will have weights to a `[5x5x3`] region in the input volume, for a total of `5*5*3 = 75` weights (and +1 bias parameter). Notice that the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.

*Example 2.*
Suppose an input volume had size \[16x16x20\]. Then using an example receptive field size of 3x3, every neuron in the Conv Layer would now have a total of `3*3*20 = 180` connections to the input volume. Notice that, again, the connectivity is local in space (e.g. 3x3), but full along the input depth (20).

A|B
:-:|:-:
![](./Piks/depthcol.jpeg)|![](./Piks/neuron_model.jpeg)

**Left**: *An example input volume in red (e.g. a 32x32x3 CIFAR-10 image), and an example volume of neurons in the first Convolutional layer. Each neuron in the convolutional layer is connected **only to a local region** in the input volume spatially, but to the **full depth** (i.e. all color channels). Note, there are multiple neurons (5 in this example) along the depth, all looking at the same region in the input - see discussion of depth columns in text below.*

**Right**: *The neurons from the Neural Network chapter remain unchanged: They still compute a dot product of their weights with the input followed by a non-linearity, but their connectivity is now restricted to be **local spatially**.*

**Spatial arrangement.**
<br>We have explained the connectivity of each neuron in the Conv Layer to the input volume, but we haven’t yet discussed *how many neurons there are in the output volume or how they are arranged*. **Three** hyperparameters control the size of the output volume: **the depth, stride and zero-padding**:
1. First, **`the depth`** of the output volume is a hyperparameter: it corresponds to the number of filters we would like to use, each learning to look for something different in the input. For example, if the first Convolutional Layer takes as input the raw image, then different neurons along the depth dimension may activate in presence of various oriented edges, or blobs of color. We will refer to a set of neurons that are all looking at the same region of the input as a **depth column** (some people also prefer the term *`fibre`*).
2. Second, we must specify the **`stride`** with which we slide the filter. When the stride is 1 then we move the filters one pixel at a time. When the stride is 2 (or uncommonly 3 or more, though this is rare in practice) then the filters jump 2 pixels at a time as we slide them around. This will produce smaller output volumes spatially.
3. As we will soon see, sometimes it will be convenient to **pad the input volume with zeros around the border**. The size of this **`zero-padding`** is a hyperparameter. The nice feature of zero padding is that it will allow us to control the spatial size of the output volumes (most commonly as we’ll see soon we will use it to exactly preserve the spatial size of the input volume so the input and output width and height are the same).

We can compute the spatial size of the output volume as a function of:
- The input volume size $(W)$, 
- The receptive field size of the Conv Layer neurons $(F)$, 
- The stride with which they are applied $(S)$, 
- The amount of zero padding used $(P)$ on the border.

You can convince yourself that the correct formula for calculating how many neurons "fit" is given by: 

$$\frac{(W−F+2P)}{S}+1$$

For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output ($\frac{7-3+2 \times 0}{1}+1=5$). 
<br>With stride 2 we would get a 3x3 output ($\frac{7-3+2 \times 0}{2}+1=3$). 
<br>Lets also see one more graphical example:

![](./Piks/stride.jpeg)

*Illustration of spatial arrangement.*
- In this example there is only one spatial dimension (x-axis), 
- One neuron with a receptive field size of F = 3, 
- The input size is W = 5, 
- There is zero padding of P = 1.

**Left**: The neuron strided across the input in stride of S = 1, giving output of size (5 - 3 + 2)/1+1 = 5. 
<br>**Right**: The neuron uses stride of S = 2, giving output of size (5 - 3 + 2)/2+1 = 3. Notice that stride S = 3 could not be used since it wouldn't fit neatly across the volume. In terms of the equation, this can be determined since (5 - 3 + 2) = 4 is not divisible by 3.
The neuron weights are in this example \[1,0,-1\] (shown on very right), and its bias is zero. These weights are shared across all yellow neurons (see parameter sharing below).

*Use of zero-padding.* In the example above on left, note that the input dimension was 5 and the output dimension was equal: also 5. This worked out so because our receptive fields were 3 and we used zero padding of 1. If there was no zero-padding used, then the output volume would have had spatial dimension of only 3, because that is how many neurons would have "fit" across the original input. In general, setting zero padding to be $P=(F−1)/2$ when the stride is $S=1$ ensures that the input volume and output volume will have the same size spatially. It is very common to use zero-padding in this way and we will discuss the full reasons when we talk more about ConvNet architectures.

*Constraints on strides.* Note again that the spatial arrangement hyperparameters have mutual constraints. For example, when the input has size $W=10$, no zero-padding is used $P=0$, and the filter size is $F=3$, then it would be impossible to use stride $S=2$, since $(W−F+2P)/S+1=(10−3+0)/2+1=4.5$, i.e. not an integer, indicating that the neurons don’t "fit" neatly and symmetrically across the input. Therefore, this setting of the hyperparameters is considered to be invalid, and a ConvNet library could throw an exception or zero pad the rest to make it fit, or crop the input to make it fit, or something. As we will see in the ConvNet architectures section, sizing the ConvNets appropriately so that all the dimensions "work out" can be a real headache, which the use of zero-padding and some design guidelines will significantly alleviate.

*Real-world example.* [The Krizhevsky et al.](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks) architecture that won the ImageNet challenge in 2012 accepted images of size \[227x227x3\]. On the first Convolutional Layer, it used neurons with receptive field size $F=11$, stride $S=4$ and no zero padding $P=0$. Since (227 - 11)/4 + 1 = 55, and since the Conv layer had a depth of $K=96$, the Conv layer output volume had size \[55x55x96\]. Each of the 55\*55\*96 neurons in this volume was connected to a region of size \[11x11x3\] in the input volume. 
<br>Moreover, all 96 neurons in each depth column are connected to the same \[11x11x3\] region of the input, but of course with different weights. As a fun aside, if you read the actual paper it claims that the input images were 224x224, which is surely incorrect because (224 - 11)/4 + 1 is quite clearly not an integer. This has confused many people in the history of ConvNets and little is known about what happened. My own best guess is that Alex used zero-padding of 3 extra pixels that he does not mention in the paper.

**Parameter Sharing.** Parameter sharing scheme is used in Convolutional Layers to control the number of parameters. Using the real-world example above, we see that:
- There are 55\*55\*96 = 290,400 neurons in the first Conv Layer. 
- Each has 11\*11\*3 = 363 weights and 1 bias. 
- Together, this adds up to 290400 \* 364 = 105,705,600 parameters on the first layer of the ConvNet alone. Clearly, this number is very high.

It turns out that we can dramatically reduce the number of parameters by making one reasonable assumption: That **if one feature is useful to compute at some spatial position (x, y), then it should also be useful to compute at a different position (x2, y2)**. 
<br>In other words, denoting a single 2-dimensional slice of depth as a depth slice (e.g. a volume of size \[55x55x96\] has 96 depth slices, each of size \[55x55\]), we are going to constrain the neurons in each depth slice to use the same weights and bias. With this parameter sharing scheme, the first Conv Layer in our example would now have only 96 unique set of weights (one for each depth slice), for a total of 96\*11\*11\*3 = 34,848 unique weights, or 34,944 parameters (+96 biases). 
<br>Alternatively, all 55\*55 neurons in each depth slice will now be using the same parameters. In practice during backpropagation, every neuron in the volume will compute the gradient for its weights, but these gradients will be added up across each depth slice and only update a single set of weights per slice.

Notice that if all neurons in a single depth slice are using the same weight vector, then the forward pass of the CONV layer can in each depth slice be computed as a convolution of the neuron’s weights with the input volume (Hence the name: Convolutional Layer). This is why it is common to refer to the *sets of weights* as a **filter** (or a **kernel**), that is convolved with the input.

![](./Piks/weights.jpeg)

*Example filters learned by Krizhevsky et al. Each of the 96 filters shown here is of size \[11x11x3\], and each one is shared by the 55x55 neurons in one depth slice. Notice that the parameter sharing assumption is relatively reasonable: If detecting a horizontal edge is important at some location in the image, it should intuitively be useful at some other location as well due to the translationally-invariant structure of images. There is therefore no need to relearn to detect a horizontal edge at every one of the 55x55 distinct locations in the Conv layer output volume.*

Note that sometimes the parameter sharing assumption may not make sense. This is especially the case when the input images to a ConvNet have some specific centered structure, where we should expect, for example, that completely different features should be learned on one side of the image than another. One practical example is when the input are faces that have been centered in the image. You might expect that different eye-specific or hair-specific features could (and should) be learned in different spatial locations. In that case it is common to relax the parameter sharing scheme, and instead simply call the layer a **Locally-Connected Layer**.

continue later